In [1]:
import os
import sys
import numpy as np
import pydicom
import dicom2nifti
from PIL import Image

In [ ]:
print(os.getcwd())
!mkdir -p ./assets/niftynet_masked_images

In [ ]:
curdir = os.getcwd()
dicom_dir = "/Users/eiofinova/niftynet/TestDICOMsHealthy/"
print(os.listdir(dicom_dir))
for x in os.listdir(dicom_dir):
    print(x)
    id = x[3:-4]
    if x.endswith(".zip"):
        !cd {dicom_dir}; mkdir "tmp"; mkdir {id}
        !cd {dicom_dir}; unzip "{x}" -d tmp;
        !cd {dicom_dir}; mv tmp/*/*/*.dcm {id}; rm -r "tmp"
        !cd {curdir}
        

In [ ]:
dicom_dir = "/Users/eiofinova/niftynet/TestDICOMsHealthy/"
sessions = [x for x in os.listdir(dicom_dir) if not x.startswith(".") and not x.endswith("zip")]
print(sessions)
for session in sessions:
    print(sessions)
    #in_dir = os.path.join("/Users/eiofinova/niftynet/data/ct_dicom", names[i])
    name = session
    print(name)
    idicom_dir = os.path.join(dicom_dir, session)
    print(idicom_dir)
    in_dir = os.path.join("/Users/eiofinova/niftynet/data/ct_dicom", name)
    if not os.path.exists(in_dir):
        os.mkdir(in_dir)
    out_dir = os.path.join("/Users/eiofinova/niftynet/data/ct_nifti", name)
    #for inner in [x for x in os.listdir(os.path.join(dicom_dir, session)) if not x.startswith(".")]:
    #    idicom_dir = [os.path.join(dicom_dir, session, inner, x) for x in os.listdir(os.path.join(dicom_dir, session, inner)) if not x.startswith(".")][0]
    #    files = [x for x in os.listdir(idicom_dir) if not x.startswith(".")]
    #idicom_dir = os.path.join(dicom_dir, session, inner)
    
    files = [x for x in os.listdir(idicom_dir) if not x.startswith(".")]
    base_name = ".".join(files[0].split(".")[-1])
    files = [pydicom.dcmread(os.path.join(idicom_dir, f)) for f in os.listdir(idicom_dir) \
             if os.path.isfile(os.path.join(idicom_dir, f)) and not f.startswith(".")]

    files.sort(key=lambda x: x.SliceLocation)
    for i, file in enumerate(files):
        file.save_as(os.path.join(in_dir, base_name + ".%s.dcm" % str((i+1))))
#     for file in files:
        
#         os.rename(os.path.join(idicom_dir, file), os.path.join(in_dir, file))
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    dicom2nifti.convert_directory(
        in_dir, 
        out_dir, 
        compression=False)
    
    for ct in os.listdir(out_dir):
        os.rename(os.path.join(out_dir, ct), os.path.join("/Users/eiofinova/niftynet/data/ct_nifti",
                                                          "CT_" + name + ".nii"))
    os.rmdir(out_dir)
    
    
! ls /Users/eiofinova/niftynet/data/

In [ ]:
for file in os.listdir("/Users/eiofinova/niftynet/data/ct_nifti"):
    if len(file) > 10:
        os.remove(os.path.join("/Users/eiofinova/niftynet/data/ct_nifti", file))

In [ ]:
config_tmpl_file = "/Users/eiofinova/niftynet/extensions/dense_vnet_abdominal_ct/config_tmpl.ini"
with open(config_tmpl_file) as f:
    config_tmpl = f.readlines()
config_tmpl
lines = [re.sub('!!!!FILE!!!!','hello', x) for x in config_tmpl]
lines

In [ ]:
!net_segment inference -c ~/niftynet/extensions/dense_vnet_abdominal_ct/config.ini

In [ ]:
seg_dir = "/Users/eiofinova/niftynet/data/mask_nifti"
jpeg_output_dir = "/Users/eiofinova/niftynet/data/mask_jpg"
print(os.listdir(seg_dir))
for file in os.listdir(seg_dir):
    if file[-4:] == ".csv":
        continue
    #file = "window_seg__%s__niftynet_out.nii" % name
    print(file)
    name = file.split("__")[1] #"window_seg__1003AbrahamAllsup__niftynet_out.nii"
    !med2image -i {os.path.join(seg_dir, file)}  -d {jpeg_output_dir} -o {name + ".jpg"}  -s -1

In [ ]:
! ls /Users/eiofinova/niftynet/data/mask_nifti

In [ ]:
name = '5'

def create_images_for_display(name, reverse = False):
    path = "/Users/eiofinova/niftynet/data/ct_dicom/" + name
    mask_path = "/Users/eiofinova/niftynet/data/mask_jpg/"
    !mkdir -p {os.path.join(os.getcwd(), "assets/niftynet_masked_images_reversed", name)}
    !mkdir -p {os.path.join(os.getcwd(), "assets/niftynet_raw_images", name)}
    files = [pydicom.dcmread(os.path.join(path, f)) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    files.sort(key=lambda x: x.SliceLocation, reverse=reverse)
    #print([x.SliceLocation for x in files])
    num_slides = len(files)
    print ("num slides is ", num_slides)

    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    from skimage import io
    import io as jio

    plt.figure(figsize = (4,4))
    gs1 = gridspec.GridSpec(4, 4)
    gs1.update(wspace=0.025, hspace=0.05) # set the spacing between axes. 

    #from matplotlib.pyplot import figure
    figure(num=None, figsize=(16, 16))

    for i, file in enumerate(files):

        idx = int(i/16)
        reverse_suffix = "_reversed" if reverse else ""
        

        ext = num_slides - 1 - i
        try:
            mask = io.imread(os.path.join(mask_path, '%s-slice%s.jpg' % (name, str(ext).zfill(3))))
        except:
            print("failed to read file ", os.path.join(mask_path, '%s-slice%s.jpg' % (name, str(ext).zfill(3))) )
            continue
        # Keep only the green channel.
        mask[:,:,0] = 0
        mask[:,:,2] = 0
        orig = extract_pixels_for_viewing(file)

        m = np.ones((mask.shape[0], mask.shape[1], 1))*50
        mm = np.concatenate((mask, m), axis=2)
        mim = Image.fromarray(np.uint8(mm))
        #aim = Image.fromarray(np.uint8(255.9*a))
        aim = Image.fromarray(orig, mode='L')
        #waim = aim.convert('RGB')
        aim.save(os.path.join(os.getcwd(), "assets/niftynet_raw_images", name, str(i)+".png"), format='PNG')
        rgbimg = Image.new("RGBA", aim.size)
        rgbimg.paste(aim)
        rgbimg.paste(mim, (0, 0), mim)
        

        output = "/tmp/test_%d.dcm" % i
        ds = file
        rgbimg.save(os.path.join(os.getcwd(), "assets/niftynet_masked_images" + reverse_suffix, name, str(i)+".png"), format='PNG')
        if i % 16 == 0 and idx < 16:
            rgbimg.show()
            ax1 = plt.subplot(gs1[idx])
            plt.axis('off')
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.set_aspect('equal')
            plt.subplots_adjust(wspace=None, hspace=None)
            plt.imshow(rgbimg)
    plt.savefig("assets/sample_crosssections%s/" % reverse_suffix + name + ".png", format="png")
    
create_images_for_display("36")

In [ ]:
for file in os.listdir("/Users/eiofinova/niftynet/data/mask_nifti/"):
    print(file)
    create_images_for_display(file.split("__")[1]) #window_seg__56__niftynet_out.nii
    

In [ ]:
myfile = pydicom.read_file("/Users/eiofinova/niftynet/data/ct_dicom/37/d.c.m.92.dcm")
dir(myfile)

In [ ]:
import matplotlib.pyplot as plt
# HU = grey values * slope + intercept
#These HU values have to be converted to pixel intensity f(x,y) using
#f(x,y) = ((HU-P1)*pow(2,i-1))/W, where P1 is the window left border,
#i is the radiometric resolution and W is the window width. 


def extract_pixels_for_viewing(dicom):
    pixels = dicom.pixel_array
    hu = pixels * dicom.RescaleSlope + dicom.RescaleIntercept
    return win_scale(hu, 60, 400, np.uint8, [0, 255])
Image.fromarray(jj, mode='L')

In [ ]:
pixels

In [ ]:
win_dict = {'abdomen':
            {'wl': 60, 'ww': 400},
            'angio':
            {'wl': 300, 'ww': 600},
            'bone':
            {'wl': 300, 'ww': 1500},
            'brain':
            {'wl': 40, 'ww': 80},
            'chest':
            {'wl': 40, 'ww': 400},
            'lungs':
            {'wl': -400, 'ww': 1500}}

def win_scale(data, wl, ww, dtype, out_range):
    """
    Scale pixel intensity data using specified window level, width, and intensity range.
    """
    
    data_new = np.empty(data.shape, dtype=np.double)
    data_new.fill(out_range[1]-1)
    
    data_new[data <= (wl-ww/2.0)] = out_range[0]
    data_new[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))] = \
         ((data[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))]-(wl-0.5))/(ww-1.0)+0.5)*(out_range[1]-out_range[0])+out_range[0]
    data_new[data > (wl+ww/2.0)] = out_range[1]-1
    
    return data_new.astype(dtype)

#aim = Image.fromarray(np.uint8(255.9*a))
jj = win_scale(orig, 60, 4000, np.uint8, [0, 255])
Image.fromarray(jj)

In [ ]:
for file in os.listdir("/Users/eiofinova/niftynet/data/mask_nifti"):
    if file.endswith("nii"):
        name = file.split("__")[1]
        if name.startswith("100"):
            create_images_for_display(name)

In [ ]:

print(a.__class__)
m = np.ones((mask.shape[0], mask.shape[1], 1))*45
mm = np.concatenate((mask, m), axis=2)
print(mm.shape)
print(mm)
mim = Image.fromarray(np.uint8(mm))
aim = Image.fromarray(np.uint8(255.9*a))
aim.paste(mim, (0, 0), mim)
aim.show()

In [ ]:
from pydicom.data import get_testdata_files
fpath = get_testdata_files("MR-SIEMENS-DICOM-WithOverlays.dcm")[0]
ds = pydicom.dcmread(fpath)
elem = ds[0x6000, 0x3000]  # returns a DataElement
print(elem)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

#plt.imsave('filename.png', np.array(result).reshape(1280,960), cmap=cm.gray)
plt.imshow(np.array(ds.pixel_array), cmap='gray')

In [ ]:
from skimage import io
image = io.imread('/Users/eiofinova/niftynet/out2/test_out-slice220.jpg')
plt.imshow(image)

In [ ]:
image[:,:,0] = 0
image[:,:,2] = 0
thresholder = lambda t: t if t < 50 else 255
vfunc = np.vectorize(thresholder)
image = vfunc(image)
image
#image = np.concatenate((image, np.ones((512,512, 1))*0.5), axis=2)

In [ ]:
plt.hist(image.ravel())

In [ ]:
plt.imshow(image)

In [ ]:
orig = ds.pixel_array
print(orig.shape)
a = np.expand_dims(orig, axis=2)
a = np.concatenate((a, a, a, a), axis=2)
a = a/2550.0
a[:, :, 3] = 1

In [ ]:
plt.imshow(a)
plt.imshow(image, alpha=0.07)
plt.show

In [ ]:
name = 'lima'
path = "/Users/eiofinova/niftynet/data/ct_dicom/" + name
files = [pydicom.dcmread(os.path.join(path, f)) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
files.sort(key=lambda x: x.SliceLocation, reverse=True)
num_slides = len(files)

In [ ]:
import re
dirs = ["niftynet_masked_images", "niftynet_masked_images_reversed", "niftynet_raw_images"]
path = "/Users/eiofinova/niftynet/assets/"

for dirr in dirs:
    for subdir in os.listdir(os.path.join(path, dirr)):
        number = re.findall(r'\d+', subdir)
        if len(number) > 0:
            #print("hello")
            ! mv {os.path.join(path, dirr, subdir)} {os.path.join(path, dirr, str(number[0]))}
        else:
            print(subdir)
        
        
        
        
        

In [ ]:
import re
dirs = ["/Users/eiofinova/niftynet/data/mask_jpg/"]
#path = "/Users/eiofinova/niftynet/assets/"

for dirr in dirs:
    for subdir in os.listdir(os.path.join(dirr)):
        parts = subdir.split("-")
        if len(parts) < 2:
            print(dirr)
            continue
        number = re.findall(r'\d+', parts[0])
        if len(number) > 0:
            #print("hello")
            ! mv {os.path.join(dirr, subdir)} {os.path.join(dirr, str(number[0]) + parts[1])}
        else:
            print(subdir)
        

In [ ]:
import re
dirs = ["sample_crosssections", "sample_crosssections_reversed"]
path = "/Users/eiofinova/niftynet/assets/"

for dirr in dirs:
    for subdir in os.listdir(os.path.join(path, dirr)):
        number = re.findall(r'\d+', subdir)
        if len(number) > 0:
            #print("hello")
            ! mv {os.path.join(path, dirr, subdir)} {os.path.join(path, dirr, str(number[0])+".png")}
        else:
            print(subdir)
        

In [ ]:
dir()